In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms, models
import glob, os

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

In [ ]:
import random

# Paths
Forest_Drive = "./Data/Forest"
NonForest_Drive = "./Data/Non_Forest"

forest_images = glob.glob(os.path.join(Forest_Drive, "**/*.tif"))
nonforest_images = glob.glob(os.path.join(NonForest_Drive, "**/*.tif"))

print(f"Total forest images found: {len(forest_images)}")
print(f"Total non-forest images found: {len(nonforest_images)}")

# Sample 2000 images from each, if possible
forest_sample = random.sample(forest_images, min(2000, len(forest_images)))
nonforest_sample = random.sample(nonforest_images, min(2000, len(nonforest_images)))

print(f"Sampled {len(forest_sample)} forest images")
print(f"Sampled {len(nonforest_sample)} non-forest images")

# Labels: Forest=1, NonForest=0
image_path = forest_sample + nonforest_sample
labels = [1]*len(forest_images) + [0]*len(nonforest_images)
print(len(image_path))


In [ ]:
import rasterio
import torch

class CustomGeoDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        # Read 4-band image with rasterio
        with rasterio.open(img_path) as src:
            image = src.read()  # shape: (bands, height, width)

        # Normalize to 0-1
        image = image.astype('float32')
        for b in range(image.shape[0]):
            band = image[b]
            band = (band - band.min()) / (band.max() - band.min() + 1e-8)
            image[b] = band

        image = torch.tensor(image, dtype=torch.float32)

        # Optional: apply transforms (resize, etc.)
        # Optional resize
        if self.transform:
            image = self.transform(image)
        else:
            import torch.nn.functional as F
            image = F.interpolate(image.unsqueeze(0), size=(380,380), mode='bilinear', align_corners=False).squeeze(0)

        return image, torch.tensor(label, dtype=torch.long)

In [ ]:
dataset = CustomGeoDataset(image_path, labels)

# Train/test split
split_size = 0.8
train_size = int(split_size * len(dataset))
test_size = len(dataset) - train_size
train_data, test_data = torch.utils.data.random_split(dataset, [train_size, test_size])

trainloader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=8, shuffle=False)

In [ ]:
from torchvision import models
import torch.nn as nn
import torch

model = models.efficientnet_b4(weights="IMAGENET1K_V1")

# original first conv
orig_conv = model.features[0][0]   # Conv2d(3,48,...)

# create new conv with 4 input channels
new_conv = nn.Conv2d(
    in_channels=4,
    out_channels=orig_conv.out_channels,
    kernel_size=orig_conv.kernel_size,
    stride=orig_conv.stride,
    padding=orig_conv.padding,
    bias=orig_conv.bias is not None
)

# copy weights for first 3 channels
with torch.no_grad():
    new_conv.weight[:, :3, :, :] = orig_conv.weight
    new_conv.weight[:, 3:, :, :] = torch.randn_like(new_conv.weight[:, 3:, :, :]) * 0.01

# replace
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)  # 1 output
model.features[0][0] = new_conv

model = model.to(DEVICE)


In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

num_epochs = 50  # adjust as needed

history = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    all_labels_train = []
    all_preds_train = []

    # -------------------
    # Training loop
    # -------------------
    for i, (x, y) in enumerate(trainloader):
        x = x.to(DEVICE, dtype=torch.float32)
        y = y.to(DEVICE, dtype=torch.float32).unsqueeze(1)  # shape [batch,1]

        optimizer.zero_grad()
        yhat = model(x)
        loss = criterion(yhat, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        preds = (torch.sigmoid(yhat) > 0.5).int()
        all_preds_train.extend(preds.detach().cpu().numpy().flatten())
        all_labels_train.extend(y.detach().cpu().numpy().flatten())

    # Training metrics
    train_acc = accuracy_score(all_labels_train, all_preds_train)
    train_prec = precision_score(all_labels_train, all_preds_train, zero_division=0)
    train_rec = recall_score(all_labels_train, all_preds_train, zero_division=0)
    train_f1 = f1_score(all_labels_train, all_preds_train, zero_division=0)

    # -------------------
    # Evaluation on test data
    # -------------------
    model.eval()
    all_labels_test = []
    all_preds_test = []
    with torch.no_grad():
        for x_test, y_test in testloader:
            x_test = x_test.to(DEVICE, dtype=torch.float32)
            y_test = y_test.to(DEVICE, dtype=torch.float32).unsqueeze(1)

            yhat_test = model(x_test)
            preds_test = (torch.sigmoid(yhat_test) > 0.5).int()

            all_preds_test.extend(preds_test.cpu().numpy().flatten())
            all_labels_test.extend(y_test.cpu().numpy().flatten())

    test_acc = accuracy_score(all_labels_test, all_preds_test)
    test_prec = precision_score(all_labels_test, all_preds_test, zero_division=0)
    test_rec = recall_score(all_labels_test, all_preds_test, zero_division=0)
    test_f1 = f1_score(all_labels_test, all_preds_test, zero_division=0)

    # Print progress
    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Train Loss: {running_loss/len(trainloader):.4f} | "
          f"Train Acc: {train_acc:.4f} | Train Prec: {train_prec:.4f} | "
          f"Train Rec: {train_rec:.4f} | Train F1: {train_f1:.4f} | "
          f"Test Acc: {test_acc:.4f} | Test Prec: {test_prec:.4f} | "
          f"Test Rec: {test_rec:.4f} | Test F1: {test_f1:.4f}")

    # -------------------
    # Save metrics to history
    # -------------------
    history.append({
        "epoch": epoch + 1,
        "train_loss": running_loss / len(trainloader),
        "train_acc": train_acc,
        "train_prec": train_prec,
        "train_rec": train_rec,
        "train_f1": train_f1,
        "test_acc": test_acc,
        "test_prec": test_prec,
        "test_rec": test_rec,
        "test_f1": test_f1
    })

# -------------------
# Save to CSV
# -------------------
df_history = pd.DataFrame(history)
df_history.to_csv("train_test_metrics.csv", index=False)
print("Saved metrics to train_test_metrics.csv")


In [ ]:
# --- After training ---
model_path = "./efficientb4_2k_forest_nonforest.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

In [ ]:
# Inspect features
print(model.features[-1])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

def contrast_stretch(img, low_pct=2, high_pct=98):
    """
    Apply percentile-based contrast stretch on a 3-channel image.
    img: np.array of shape (H,W,C), float or int
    """
    out = np.zeros_like(img, dtype=np.float32)
    for c in range(img.shape[2]):
        low, high = np.percentile(img[:,:,c], (low_pct, high_pct))
        out[:,:,c] = np.clip((img[:,:,c] - low) / (high - low + 1e-8), 0, 1)
    return out

In [ ]:
# --- GradCAM on a test image ---
target_layers = [model.features[-1][0]]  # last Conv2d

# pick one test sample
test_img, test_label = test_data[0]
input_tensor = test_img.unsqueeze(0).to(DEVICE)

# --- Prepare 321 composite ---
rgb_img_321 = test_img[[2,1,0]].numpy()  # bands 3,2,1
rgb_img_321 = np.transpose(rgb_img_321, (1,2,0))  # (H,W,C)

# --- Contrast stretch ---
rgb_img_321_stretched = contrast_stretch(rgb_img_321, 2, 98)

with GradCAM(model=model, target_layers=target_layers) as cam:
    grayscale_cam = cam(input_tensor=input_tensor)
    grayscale_cam_norm = (grayscale_cam[0] - grayscale_cam[0].min()) / (grayscale_cam[0].max() - grayscale_cam[0].min() + 1e-8) 

cam_mask = grayscale_cam[0] if grayscale_cam.ndim==3 else grayscale_cam
cam_mask_uint8 = np.uint8(255 * cam_mask)

cam_image = show_cam_on_image(rgb_img_321_stretched, cam_mask_uint8, use_rgb=True)

# --- Create heatmap with colormap ---
heatmap = cm.jet(grayscale_cam_norm)[:, :, :3]  # convert to RGB

# Overlay CAM on RGB image
cam_overlay = 0.5 * rgb_img_321_stretched + 0.5 * heatmap
cam_overlay = np.clip(cam_overlay, 0, 1)

# --- Plot side by side ---
fig, axes = plt.subplots(1, 2, figsize=(12,6))

# Original image
axes[0].imshow(rgb_img_321_stretched)
axes[0].set_title("Original Image (321 Composite)")
axes[0].axis("off")

# Grad-CAM overlay
im = axes[1].imshow(cam_overlay)
axes[1].set_title(f"Grad-CAM Overlay\nTrue Label: {int(test_label.item())}")
axes[1].axis("off")

# Add colorbar for activation intensity
cbar = fig.colorbar(cm.ScalarMappable(cmap='jet'), ax=axes[1], fraction=0.046, pad=0.04)
cbar.set_label('Activation Intensity', rotation=270, labelpad=15)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import cv2
from scipy import ndimage
from PIL import Image, ImageDraw

def extract_regions_from_cam(cam_mask, threshold=0.2, min_area=1000):
    """
    cam_mask: 2D array normalized 0..1 (grayscale_cam_norm or similar)
    returns: labeled_mask, list of region dicts {label, area, bbox, centroid, coords_mask}
    """
    bw = (cam_mask >= threshold).astype(np.uint8)
    # remove tiny islands
    bw = ndimage.binary_opening(bw, structure=np.ones((3,3))).astype(np.uint8)
    labeled, n = ndimage.label(bw)
    regions = []
    for lab in range(1, n+1):
        coords = np.argwhere(labeled == lab)  # rows (y), cols (x)
        area = coords.shape[0]
        if area < min_area:
            continue
        ys = coords[:,0]
        xs = coords[:,1]
        cy = ys.mean()
        cx = xs.mean()
        y0, x0, y1, x1 = ys.min(), xs.min(), ys.max(), xs.max()
        regions.append({
            "label": lab,
            "area": int(area),
            "bbox": (int(x0), int(y0), int(x1), int(y1)),
            "centroid": (float(cx), float(cy)),
            "coords": coords
        })
    return labeled, regions

def bezier_like_polygon_points(center, area_pixels, img_shape, n_points=8, 
                               min_scale=0.6, max_scale=1.3, jitter_angle=0.5):
    """
    Create polygon points around center. radius scales with sqrt(area).
    center: (cx, cy) in pixel coordinates (x,y)
    area_pixels: area (number of pix) of the high-activation region
    img_shape: (H, W)
    returns: list of (x,y) float points
    """
    H, W = img_shape
    cx, cy = center
    # base radius from area: radius that would have same area if circular
    base_radius = np.sqrt(area_pixels / np.pi)
    # scale to be larger a bit
    mean_radius = max(4.0, base_radius * 1.5)  # at least a few px
    angles = np.linspace(0, 2*np.pi, n_points, endpoint=False)
    angles += np.random.uniform(-jitter_angle, jitter_angle, size=n_points)
    radii = np.random.uniform(min_scale, max_scale, size=n_points) * mean_radius
    xs = cx + radii * np.cos(angles)
    ys = cy + radii * np.sin(angles)
    pts = np.stack([xs, ys], axis=1)
    # clip to image bounds
    pts[:,0] = np.clip(pts[:,0], 0, W-1)
    pts[:,1] = np.clip(pts[:,1], 0, H-1)
    return pts.tolist()

def smooth_polygon(points, smoothing_radius=3):
    """
    Given polygon points as list[(x,y)], upsample and apply Gaussian smoothing to coordinates
    to create a smoother curve. Returns list[(x,y)] int suitable for rasterization.
    """
    pts = np.array(points, dtype=np.float32)
    # close loop
    pts_closed = np.vstack([pts, pts[0:1,:]])
    # param t along polygon
    t = np.linspace(0, 1, len(pts_closed))
    # upsample
    t_up = np.linspace(0, 1, max(64, len(pts_closed)*8))
    xs = np.interp(t_up, t, pts_closed[:,0])
    ys = np.interp(t_up, t, pts_closed[:,1])
    # gaussian smooth
    k = max(1, int(smoothing_radius))
    xs = cv2.GaussianBlur(xs.reshape(1,-1).astype(np.float32), (1, k|1), sigmaX=smoothing_radius).flatten()
    ys = cv2.GaussianBlur(ys.reshape(1,-1).astype(np.float32), (1, k|1), sigmaX=smoothing_radius).flatten()
    poly = np.stack([xs, ys], axis=1)
    # convert to ints for rasterization
    poly_int = [(int(round(x)), int(round(y))) for x,y in poly]
    return poly_int

def polygon_to_mask(poly_pts, img_shape, soften_sigma=3):
    """
    Rasterize polygon points to a float mask [0,1], optionally soften with gaussian blur
    poly_pts: list of (x,y) ints
    """
    H, W = img_shape
    mask = np.zeros((H, W), dtype=np.uint8)
    if len(poly_pts) >= 3:
        cv2.fillPoly(mask, [np.array(poly_pts, dtype=np.int32)], color=1)
    mask = mask.astype(np.float32)
    if soften_sigma > 0:
        k = max(3, int(soften_sigma*4)|1)
        mask = cv2.GaussianBlur(mask, (k,k), soften_sigma)
        # normalize to 0..1
        if mask.max() > 0:
            mask = mask / mask.max()
    return mask

# ---------------------------
# Example usage (after computing grayscale_cam and rgb_img_321_stretched)
# grayscale_cam_norm should be 2D in 0..1 (H,W)
# ---------------------------

# assume you have:
# grayscale_cam_norm  (H,W) float in [0,1]
# rgb_img_321_stretched (H,W,3) float in [0,1]

threshold = 0.8
min_area = 10  # min pixels for region to be considered
labeled, regions = extract_regions_from_cam(grayscale_cam_norm, threshold=threshold, min_area=min_area)

masks = []
polygons = []

H, W = grayscale_cam_norm.shape

for r in regions:
    centroid = r["centroid"]  # (cx, cy) in pixels (note: x,y)
    area = r["area"]
    # Determine number of control/sample points depending on area
    n_points = 6 + int(min(12, max(0, np.log1p(area)/0.5)))  # between 6 and ~12
    base_pts = bezier_like_polygon_points(center=centroid, area_pixels=area, img_shape=(H,W), n_points=n_points)
    poly_smooth = smooth_polygon(base_pts, smoothing_radius=max(2, int(np.sqrt(area)/6)))
    mask = polygon_to_mask(poly_smooth, (H,W), soften_sigma=max(1.0, np.sqrt(area)/8.0))
    masks.append(mask)
    polygons.append(poly_smooth)

# If you want a combined mask with multiple regions (logical OR)
if len(masks) > 0:
    combined_mask = np.clip(np.sum(np.stack(masks, axis=0), axis=0), 0, 1)
else:
    combined_mask = np.zeros_like(grayscale_cam_norm)

# Visualization example: show mask outlines on image
vis = (rgb_img_321_stretched.copy()*255).astype(np.uint8)
for poly in polygons:
    cv2.polylines(vis, [np.array(poly, dtype=np.int32)], isClosed=True, color=(255,0,0), thickness=2)

plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.imshow(rgb_img_321_stretched)
plt.title("321 composite (stretched)")
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(vis)
plt.title("polygons from CAM regions")
plt.axis('off')
plt.show()

# masks: list of float masks [H,W] in 0..1 (soft edges)
# polygons: list of list[(x,y)] integer polygon coordinates
# combined_mask: aggregate mask (0..1)


In [ ]:
import numpy as np
from scipy.special import binom
import matplotlib.pyplot as plt

bernstein = lambda n, k, t: binom(n, k) * t**k * (1.-t)**(n-k)

def bezier(points, num=200):
    N = len(points)
    t = np.linspace(0, 1, num=num)
    curve = np.zeros((num, 2))
    for i in range(N):
        curve += np.outer(bernstein(N - 1, i, t), points[i])
    return curve

class Segment():
    def __init__(self, p1, p2, angle1, angle2, **kw):
        self.p1 = p1
        self.p2 = p2
        self.angle1 = angle1
        self.angle2 = angle2
        self.numpoints = kw.get("numpoints", 100)
        r = kw.get("r", 0.3)
        d = np.sqrt(np.sum((self.p2 - self.p1)**2))
        self.r = r * d
        self.p = np.zeros((4, 2))
        self.p[0, :] = self.p1[:]
        self.p[3, :] = self.p2[:]
        self.calc_intermediate_points(self.r)

    def calc_intermediate_points(self, r):
        self.p[1, :] = self.p1 + np.array([self.r * np.cos(self.angle1),
                                           self.r * np.sin(self.angle1)])
        self.p[2, :] = self.p2 + np.array([self.r * np.cos(self.angle2 + np.pi),
                                           self.r * np.sin(self.angle2 + np.pi)])
        self.curve = bezier(self.p, self.numpoints)

def get_curve(points, **kw):
    segments = []
    for i in range(len(points) - 1):
        seg = Segment(points[i, :2], points[i + 1, :2], points[i, 2], points[i + 1, 2], **kw)
        segments.append(seg)
    curve = np.concatenate([s.curve for s in segments])
    return segments, curve

def ccw_sort(p):
    d = p - np.mean(p, axis=0)
    s = np.arctan2(d[:, 0], d[:, 1])
    return p[np.argsort(s), :]

def get_bezier_curve(a, rad=0.2, edgy=0):
    """ given an array of points *a*, create a curve through
    those points. 
    *rad* is a number between 0 and 1 to steer the distance of
          control points.
    *edgy* is a parameter which controls how "edgy" the curve is,
           edgy=0 is smoothest."""
    p = np.arctan(edgy) / np.pi + .5
    a = ccw_sort(a)
    a = np.append(a, np.atleast_2d(a[0, :]), axis=0)
    d = np.diff(a, axis=0)
    ang = np.arctan2(d[:, 1], d[:, 0])
    f = lambda ang: (ang >= 0) * ang + (ang < 0) * (ang + 2 * np.pi)
    ang = f(ang)
    ang1 = ang
    ang2 = np.roll(ang, 1)
    ang = p * ang1 + (1 - p) * ang2 + (np.abs(ang2 - ang1) > np.pi) * np.pi
    ang = np.append(ang, [ang[0]])
    a = np.append(a, np.atleast_2d(ang).T, axis=1)
    s, c = get_curve(a, r=rad, method="var")
    x, y = c.T
    return x, y, a

def get_random_points(n=5, scale=0.8, mindst=None, rec=0):
    """ create n random points in the unit square, which are *mindst*
    apart, then scale them."""
    mindst = mindst or .7 / n
    a = np.random.rand(n, 2)
    d = np.sqrt(np.sum(np.diff(ccw_sort(a), axis=0), axis=1)**2)
    if np.all(d >= mindst) or rec >= 200:
        return a * scale
    else:
        return get_random_points(n=n, scale=scale, mindst=mindst, rec=rec + 1)

In [ ]:
from PIL import Image, ImageDraw, ImageFilter
import numpy as np
import cv2

def cam_region_to_bezier_mask(cam_mask, threshold=0.8, rad=0.3, edgy=0.2, blur_radius=5):
    """
    Convert high-activation CAM regions into soft Bezier masks.
    cam_mask: 2D np.array normalized [0,1]
    threshold: activation threshold
    blur_radius: Gaussian blur radius for soft boundaries
    Convert high-activation CAM regions into Bezier masks (clipped to image size, with optional blur).
    Returns: list of (polygon_points, polygon_mask)
    """
    H, W = cam_mask.shape
    bw = (cam_mask >= threshold).astype(np.uint8)

    # Find contours
    contours, _ = cv2.findContours(bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    results = []

    for cnt in contours:
        if len(cnt) < 5:
            continue
        pts = cnt.squeeze()
        if pts.ndim != 2 or pts.shape[0] < 5:
            continue

        # Convex hull for smoother boundary
        hull = cv2.convexHull(pts).squeeze().astype(np.float32)

        # --- Get Bezier curve ---
        x, y, _ = get_bezier_curve(hull, rad=rad, edgy=edgy)

        # --- Clip coordinates to valid image bounds ---
        x = np.clip(x, 0, W - 1)
        y = np.clip(y, 0, H - 1)

        # --- Round to int for polygon rasterization ---
        poly_points = [(int(xi), int(yi)) for xi, yi in zip(x, y)]

        # --- Create polygon mask ---
        img_mask = Image.new("L", (W, H), 0)
        ImageDraw.Draw(img_mask).polygon(poly_points, outline=1, fill=1)
        poly_mask = np.array(img_mask, dtype=np.uint8)

        # --- Apply Gaussian blur (optional feathering) ---
        if blur_radius > 0:
            poly_mask = cv2.GaussianBlur(poly_mask.astype(np.float32), 
                                         (0, 0), blur_radius)
            poly_mask = np.clip(poly_mask, 0, 1)  # normalize back to [0,1]

        results.append((poly_points, poly_mask))

    return results

In [ ]:
# ---------------- Example usage ----------------
results = cam_region_to_bezier_mask(grayscale_cam_norm, threshold=0.4, rad=0.4, edgy=0.3, blur_radius=8)

# Combine masks (clipped automatically)
H, W = grayscale_cam_norm.shape
mask_total = np.zeros((H, W), dtype=np.uint8)
for _, poly_mask in results:
    mask_total = np.maximum(mask_total, poly_mask)

# --- Visualization ---
plt.figure(figsize=(18,6))

plt.subplot(1,3,1)
plt.imshow(rgb_img_321_stretched)
plt.title("Original Image")

plt.subplot(1,3,2)
plt.imshow(rgb_img_321_stretched)
im = plt.imshow(grayscale_cam_norm, cmap='jet', alpha=0.5)
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.title("Grad-CAM Overlay")

plt.subplot(1,3,3)
plt.imshow(rgb_img_321_stretched)
plt.imshow(mask_total, cmap='Reds', alpha=0.4)
for poly_points, _ in results:
    plt.plot([p[0] for p in poly_points], [p[1] for p in poly_points], 'r-', linewidth=2)
plt.title("Bezier Mask Overlay (Soft Edges)")
plt.show()

In [ ]:
import numpy as np
import cv2
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

def cam_region_to_bezier_mask(cam_mask, threshold=0.8, rad=0.3, edgy=0.2):
    """
    Convert high-activation CAM regions into Bezier masks (clipped to image size).
    Returns: list of (polygon_points, polygon_mask)
    """
    H, W = cam_mask.shape
    bw = (cam_mask >= threshold).astype(np.uint8)

    # Find contours
    contours, _ = cv2.findContours(bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    results = []

    for cnt in contours:
        if len(cnt) < 5:
            continue
        pts = cnt.squeeze()
        if pts.ndim != 2 or pts.shape[0] < 5:
            continue

        # Convex hull for smoothness
        hull = cv2.convexHull(pts).squeeze().astype(np.float32)

        # Get Bezier curve
        x, y, _ = get_bezier_curve(hull, rad=rad, edgy=edgy)

        # --- Clip to valid image bounds ---
        x = np.clip(x, 0, W - 1)
        y = np.clip(y, 0, H - 1)

        # Build polygon points (integer coordinates)
        poly_points = [(int(xi), int(yi)) for xi, yi in zip(x, y)]

        # --- Rasterize polygon mask ---
        img_mask = Image.new('L', (W, H), 0)
        ImageDraw.Draw(img_mask).polygon(poly_points, outline=1, fill=1)
        poly_mask = np.array(img_mask, dtype=np.uint8)

        results.append((poly_points, poly_mask))

        '''
        # Get Bezier curve
        x, y, _ = get_bezier_curve(hull, rad=rad, edgy=edgy)
        poly_points = list(zip(x, y))

        # --- Rasterize polygon mask ---
        img_mask = Image.new('L', (W, H), 0)
        ImageDraw.Draw(img_mask).polygon(poly_points, outline=1, fill=1)
        poly_mask = np.array(img_mask, dtype=np.uint8)

        results.append((poly_points, poly_mask))
        '''

    return results

# ---------------- Example usage ----------------
results = cam_region_to_bezier_mask(grayscale_cam_norm, threshold=0.35, rad=0.8, edgy=0.5)

# Combine masks (clipped automatically)
H, W = grayscale_cam_norm.shape
mask_total = np.zeros((H, W), dtype=np.uint8)
for _, poly_mask in results:
    mask_total = np.maximum(mask_total, poly_mask)

# --- Visualization ---
plt.figure(figsize=(18,6))

# 1. Show CAM directly
plt.subplot(1,3,1)
plt.imshow(rgb_img_321_stretched)
plt.title("Original Image")

# 2. Overlay CAM on original image
plt.subplot(1,3,2)
plt.imshow(rgb_img_321_stretched)
im = plt.imshow(grayscale_cam_norm, cmap='jet', alpha=0.5)  # heatmap overlay
plt.colorbar(im, fraction=0.046, pad=0.04) 
plt.title("Grad-CAM Overlay")

# 3. Overlay Bezier masks on original image
plt.subplot(1,3,3)
plt.imshow(rgb_img_321_stretched)
plt.imshow(mask_total, cmap='Reds', alpha=0.4)  # overlay masks
for poly_points, _ in results:
    plt.plot([p[0] for p in poly_points], [p[1] for p in poly_points], 'r-', linewidth=2)
plt.title("Bezier Mask Overlay")

plt.tight_layout()
plt.show()


In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

# ----------------------
# Albumentations Transform
# ----------------------
geo_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(
        shift_limit=0.1, scale_limit=0.2, rotate_limit=30,
        interpolation=1, border_mode=0, p=0.7
    )
])

def tensor_to_numpy(image_tensor):
    """Convert tensor (C,H,W) -> numpy (H,W,C) for Albumentations."""
    return image_tensor.permute(1, 2, 0).cpu().numpy()

def numpy_to_tensor(image_np):
    """Convert numpy (H,W,C) -> tensor (C,H,W)."""
    return torch.from_numpy(image_np).permute(2, 0, 1).float()


def apply_albumentations(image_tensor, transform=geo_transform):
    """Apply Albumentations transform to a 4-band tensor image."""
    image_np = tensor_to_numpy(image_tensor)
    augmented = transform(image=image_np)
    return numpy_to_tensor(augmented["image"])

In [ ]:
import Model.salient_bezier_cutmix as sbc
print(dir(sbc))

from  Model.salient_bezier_cutmix import salient_cutmix_pipeline
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

n = 50

Target_Drive = "./Data_3/Forest" # data that will multiplied by mask
Source_Drive = "./Data_3/NonForest" # data that will multiplied by mask and added to the target drive
out_img_drive = "./Data_3/Augmented_Image"
out_label_drive = "./Data_3/Augmented_Label"

os.makedirs(out_img_drive, exist_ok=True)
os.makedirs(out_label_drive, exist_ok=True)

salient_cutmix_pipeline(
    model=model,
    target_drive=Target_Drive,
    source_drive=Source_Drive,
    out_img_drive=out_img_drive,
    out_label_drive=out_label_drive,
    n=n,
    device=DEVICE
)


In [ ]:
# Core Libraries
import os
import glob
import random

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# NumPy
import numpy as np

# OpenCV
import cv2

# Visualization
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# PIL
from PIL import Image, ImageDraw, ImageFilter

# TorchVision
from torchvision import transforms, models

# Grad-CAM
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

# SciPy
from scipy import ndimage
from scipy.special import binom

# RasterIO
import rasterio
from rasterio.transform import Affine

# Albumentations
import albumentations as A

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

class CustomGeoDataset2(torch.utils.data.Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        #label = self.labels[idx]

        # Read 4-band image with rasterio
        with rasterio.open(img_path) as src:
            image = src.read()  # shape: (bands, height, width)

        # Normalize to 0-1
        image = image.astype('float32')
        for b in range(image.shape[0]):
            band = image[b]
            band = (band - band.min()) / (band.max() - band.min() + 1e-8)
            image[b] = band

        image = torch.tensor(image, dtype=torch.float32)

        # Optional: apply transforms (resize, etc.)
        # Optional resize
        if self.transform:
            image = self.transform(image)
        else:
            image = F.interpolate(image.unsqueeze(0), size=(380,380), mode='bilinear', align_corners=False).squeeze(0)

        return image

def load_random_paths(base_dir, n):
    all_files = [os.path.join(base_dir, f) for f in os.listdir(base_dir) if f.endswith(".tif")]
    return random.sample(all_files, min(n, len(all_files)))

def load_and_preprocess(img_path, size=(256, 256)):
    """Load GeoTIFF with rasterio and normalize"""
    with rasterio.open(img_path) as src:
        img = src.read().astype("float32")  # (C, H, W)

    # Normalize band-wise
    for b in range(img.shape[0]):
        band = img[b]
        img[b] = (band - band.min()) / (band.max() - band.min() + 1e-8)

    # Resize to (C, 256, 256)
    img_tensor = torch.tensor(img).unsqueeze(0)  # (1, C, H, W)
    img_resized = F.interpolate(img_tensor, size=size, mode="bilinear", align_corners=False)
    return img_resized.squeeze(0).numpy()  # (C, H, W)

def contrast_stretch(img, low_pct=2, high_pct=98):
    """
    Apply percentile-based contrast stretch on a 3-channel image.
    img: np.array of shape (H,W,C), float or int
    """
    out = np.zeros_like(img, dtype=np.float32)
    for c in range(img.shape[2]):
        low, high = np.percentile(img[:,:,c], (low_pct, high_pct))
        out[:,:,c] = np.clip((img[:,:,c] - low) / (high - low + 1e-8), 0, 1)
    return out

def extract_regions_from_cam(cam_mask, threshold=0.2, min_area=1000):
    """
    cam_mask: 2D array normalized 0..1 (grayscale_cam_norm or similar)
    returns: labeled_mask, list of region dicts {label, area, bbox, centroid, coords_mask}
    """
    bw = (cam_mask >= threshold).astype(np.uint8)
    # remove tiny islands
    bw = ndimage.binary_opening(bw, structure=np.ones((3,3))).astype(np.uint8)
    labeled, n = ndimage.label(bw)
    regions = []
    for lab in range(1, n+1):
        coords = np.argwhere(labeled == lab)  # rows (y), cols (x)
        area = coords.shape[0]
        if area < min_area:
            continue
        ys = coords[:,0]
        xs = coords[:,1]
        cy = ys.mean()
        cx = xs.mean()
        y0, x0, y1, x1 = ys.min(), xs.min(), ys.max(), xs.max()
        regions.append({
            "label": lab,
            "area": int(area),
            "bbox": (int(x0), int(y0), int(x1), int(y1)),
            "centroid": (float(cx), float(cy)),
            "coords": coords
        })
    return labeled, regions

def bezier_like_polygon_points(center, area_pixels, img_shape, n_points=8, 
                               min_scale=0.6, max_scale=1.3, jitter_angle=0.5):
    """
    Create polygon points around center. radius scales with sqrt(area).
    center: (cx, cy) in pixel coordinates (x,y)
    area_pixels: area (number of pix) of the high-activation region
    img_shape: (H, W)
    returns: list of (x,y) float points
    """
    H, W = img_shape
    cx, cy = center
    # base radius from area: radius that would have same area if circular
    base_radius = np.sqrt(area_pixels / np.pi)
    # scale to be larger a bit
    mean_radius = max(4.0, base_radius * 1.5)  # at least a few px
    angles = np.linspace(0, 2*np.pi, n_points, endpoint=False)
    angles += np.random.uniform(-jitter_angle, jitter_angle, size=n_points)
    radii = np.random.uniform(min_scale, max_scale, size=n_points) * mean_radius
    xs = cx + radii * np.cos(angles)
    ys = cy + radii * np.sin(angles)
    pts = np.stack([xs, ys], axis=1)
    # clip to image bounds
    pts[:,0] = np.clip(pts[:,0], 0, W-1)
    pts[:,1] = np.clip(pts[:,1], 0, H-1)
    return pts.tolist()

def smooth_polygon(points, smoothing_radius=3):
    """
    Given polygon points as list[(x,y)], upsample and apply Gaussian smoothing to coordinates
    to create a smoother curve. Returns list[(x,y)] int suitable for rasterization.
    """
    pts = np.array(points, dtype=np.float32)
    # close loop
    pts_closed = np.vstack([pts, pts[0:1,:]])
    # param t along polygon
    t = np.linspace(0, 1, len(pts_closed))
    # upsample
    t_up = np.linspace(0, 1, max(64, len(pts_closed)*8))
    xs = np.interp(t_up, t, pts_closed[:,0])
    ys = np.interp(t_up, t, pts_closed[:,1])
    # gaussian smooth
    k = max(1, int(smoothing_radius))
    xs = cv2.GaussianBlur(xs.reshape(1,-1).astype(np.float32), (1, k|1), sigmaX=smoothing_radius).flatten()
    ys = cv2.GaussianBlur(ys.reshape(1,-1).astype(np.float32), (1, k|1), sigmaX=smoothing_radius).flatten()
    poly = np.stack([xs, ys], axis=1)
    # convert to ints for rasterization
    poly_int = [(int(round(x)), int(round(y))) for x,y in poly]
    return poly_int

def polygon_to_mask(poly_pts, img_shape, soften_sigma=3):
    """
    Rasterize polygon points to a float mask [0,1], optionally soften with gaussian blur
    poly_pts: list of (x,y) ints
    """
    H, W = img_shape
    mask = np.zeros((H, W), dtype=np.uint8)
    if len(poly_pts) >= 3:
        cv2.fillPoly(mask, [np.array(poly_pts, dtype=np.int32)], color=1)
    mask = mask.astype(np.float32)
    if soften_sigma > 0:
        k = max(3, int(soften_sigma*4)|1)
        mask = cv2.GaussianBlur(mask, (k,k), soften_sigma)
        # normalize to 0..1
        if mask.max() > 0:
            mask = mask / mask.max()
    return mask

bernstein = lambda n, k, t: binom(n, k) * t**k * (1.-t)**(n-k)

def bezier(points, num=200):
    N = len(points)
    t = np.linspace(0, 1, num=num)
    curve = np.zeros((num, 2))
    for i in range(N):
        curve += np.outer(bernstein(N - 1, i, t), points[i])
    return curve

class Segment():
    def __init__(self, p1, p2, angle1, angle2, **kw):
        self.p1 = p1
        self.p2 = p2
        self.angle1 = angle1
        self.angle2 = angle2
        self.numpoints = kw.get("numpoints", 100)
        r = kw.get("r", 0.3)
        d = np.sqrt(np.sum((self.p2 - self.p1)**2))
        self.r = r * d
        self.p = np.zeros((4, 2))
        self.p[0, :] = self.p1[:]
        self.p[3, :] = self.p2[:]
        self.calc_intermediate_points(self.r)

    def calc_intermediate_points(self, r):
        self.p[1, :] = self.p1 + np.array([self.r * np.cos(self.angle1),
                                           self.r * np.sin(self.angle1)])
        self.p[2, :] = self.p2 + np.array([self.r * np.cos(self.angle2 + np.pi),
                                           self.r * np.sin(self.angle2 + np.pi)])
        self.curve = bezier(self.p, self.numpoints)

def get_curve(points, **kw):
    segments = []
    for i in range(len(points) - 1):
        seg = Segment(points[i, :2], points[i + 1, :2], points[i, 2], points[i + 1, 2], **kw)
        segments.append(seg)
    curve = np.concatenate([s.curve for s in segments])
    return segments, curve

def ccw_sort(p):
    d = p - np.mean(p, axis=0)
    s = np.arctan2(d[:, 0], d[:, 1])
    return p[np.argsort(s), :]

def get_bezier_curve(a, rad=0.2, edgy=0):
    """ given an array of points *a*, create a curve through
    those points. 
    *rad* is a number between 0 and 1 to steer the distance of
          control points.
    *edgy* is a parameter which controls how "edgy" the curve is,
           edgy=0 is smoothest."""
    p = np.arctan(edgy) / np.pi + .5
    a = ccw_sort(a)
    a = np.append(a, np.atleast_2d(a[0, :]), axis=0)
    d = np.diff(a, axis=0)
    ang = np.arctan2(d[:, 1], d[:, 0])
    f = lambda ang: (ang >= 0) * ang + (ang < 0) * (ang + 2 * np.pi)
    ang = f(ang)
    ang1 = ang
    ang2 = np.roll(ang, 1)
    ang = p * ang1 + (1 - p) * ang2 + (np.abs(ang2 - ang1) > np.pi) * np.pi
    ang = np.append(ang, [ang[0]])
    a = np.append(a, np.atleast_2d(ang).T, axis=1)
    s, c = get_curve(a, r=rad, method="var")
    x, y = c.T
    return x, y, a

def get_random_points(n=5, scale=0.8, mindst=None, rec=0):
    """ create n random points in the unit square, which are *mindst*
    apart, then scale them."""
    mindst = mindst or .7 / n
    a = np.random.rand(n, 2)
    d = np.sqrt(np.sum(np.diff(ccw_sort(a), axis=0), axis=1)**2)
    if np.all(d >= mindst) or rec >= 200:
        return a * scale
    else:
        return get_random_points(n=n, scale=scale, mindst=mindst, rec=rec + 1)

def cam_region_to_bezier_mask(cam_mask, threshold=0.8, rad=0.3, edgy=0.2, blur_radius=5):
    """
    Convert high-activation CAM regions into soft Bezier masks.
    cam_mask: 2D np.array normalized [0,1]
    threshold: activation threshold
    blur_radius: Gaussian blur radius for soft boundaries
    Convert high-activation CAM regions into Bezier masks (clipped to image size, with optional blur).
    Returns: list of (polygon_points, polygon_mask)
    """
    H, W = cam_mask.shape
    bw = (cam_mask >= threshold).astype(np.uint8)

    # Find contours
    contours, _ = cv2.findContours(bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    results = []

    for cnt in contours:
        if len(cnt) < 5:
            continue
        pts = cnt.squeeze()
        if pts.ndim != 2 or pts.shape[0] < 5:
            continue

        # Convex hull for smoother boundary
        hull = cv2.convexHull(pts).squeeze().astype(np.float32)

        # --- Get Bezier curve ---
        x, y, _ = get_bezier_curve(hull, rad=rad, edgy=edgy)

        # --- Clip coordinates to valid image bounds ---
        x = np.clip(x, 0, W - 1)
        y = np.clip(y, 0, H - 1)

        # --- Round to int for polygon rasterization ---
        poly_points = [(int(xi), int(yi)) for xi, yi in zip(x, y)]

        # --- Create polygon mask ---
        img_mask = Image.new("L", (W, H), 0)
        ImageDraw.Draw(img_mask).polygon(poly_points, outline=1, fill=1)
        poly_mask = np.array(img_mask, dtype=np.uint8)

        # --- Apply Gaussian blur (optional feathering) ---
        if blur_radius > 0:
            poly_mask = cv2.GaussianBlur(poly_mask.astype(np.float32), 
                                         (0, 0), blur_radius)
            poly_mask = np.clip(poly_mask, 0, 1)  # normalize back to [0,1]

        results.append((poly_points, poly_mask))

    return results

geom_transform = A.Compose(
    [
        A.HorizontalFlip(p=0.9),
        A.VerticalFlip(p=0.9),
        A.RandomRotate90(p=0.9),
    ],
    additional_targets={
        "raw": "image",
        "target": "image"
    }
)

def apply_cam_and_mask(model, img_array, device="cuda"):
    """Run model -> CAM -> Bezier mask"""
    model.eval()
    target_layers = [model.features[-1][0]]  # last Conv2d
    img_tensor = img_array.unsqueeze(0).to(DEVICE)

    with GradCAM(model=model, target_layers=target_layers) as cam:
        gcam = cam(input_tensor=img_tensor)
        # Normalize CAM
        gcam = (gcam - gcam.min()) / (gcam.max() - gcam.min() + 1e-8)
        gcam = gcam.squeeze()
    
    # Generate mask
    masks = cam_region_to_bezier_mask(gcam, threshold=0.3, rad=0.9, edgy=0.5, blur_radius=2)

    H, W = gcam.shape
    mask_total = np.zeros((H, W), dtype=np.uint8)
    for _, poly_mask in masks:
        mask_total = np.maximum(mask_total, poly_mask)

    # Resize to 256×256
    mask_resized = cv2.resize(mask_total, (256, 256), interpolation=cv2.INTER_LINEAR)

    return mask_resized

def salient_cutmix_pipeline(model, target_drive, source_drive, out_img_drive, out_label_drive, n=500, device="cuda"):
    target_paths = load_random_paths(target_drive, n)
    source_paths = load_random_paths(source_drive, n)

    for i, (target_path, source_path) in enumerate(zip(target_paths, source_paths)):
        # --- Load normalized (for CAM) ---
        s_img = CustomGeoDataset2([source_path], transform=None)[0]  # normalized (C,H,W)
        s_img = s_img.numpy().transpose(1, 2, 0)  # HWC in [0,1]

        # --- Load unnormalized (for blending) ---
        with rasterio.open(source_path) as src:
            s2_img = src.read().transpose(1, 2, 0).astype(np.float32)  # original values
            s2_img = cv2.resize(s2_img, (380, 380), interpolation=cv2.INTER_LINEAR)
            #src_profile = src.profile

        # --- Apply SAME augmentation to both normalized + unnormalized ---
        aug_out = geom_transform(image=s_img, raw=s2_img)
        s_aimg = aug_out["image"]   # normalized → for CAM
        s2_aimg = aug_out["raw"]    # unnormalized → for blending

        # --- Saliency mask from normalized ---
        s_aimg = np.transpose(s_aimg, (2, 0, 1))
        s_tensor = torch.tensor(s_aimg, dtype=torch.float32).to(device)
        mask = apply_cam_and_mask(model, s_tensor, device=device)  # (H,W)
        mask_binary = (mask > 0.25).astype(np.uint8)

        # --- Load and augment target ---
        with rasterio.open(target_path) as tgt:
            profile = tgt.profile
            t_img = tgt.read().transpose(1, 2, 0).astype(np.float32)

        t_aimg = geom_transform(image=t_img)["image"]

        # --- Blend in original value space ---
        mask_expanded = np.expand_dims(mask, axis=-1).astype(np.float32)
        s2_aimg = cv2.resize(s2_img, (256, 256), interpolation=cv2.INTER_LINEAR)
        t_aimg = cv2.resize(t_aimg, (256, 256), interpolation=cv2.INTER_LINEAR)
        mixed_img = mask_expanded * s2_aimg + (1 - mask_expanded) * t_aimg

        # --- Prepare profiles ---
        img_profile = profile.copy()
        img_profile.update({
            "count": mixed_img.shape[2],
            "dtype": "float32"
        })

        label_profile = profile.copy()
        label_profile.update({
            "count": 1,
            "dtype": "uint8"
        })

        # --- Save results ---
        out_img_path = os.path.join(out_img_drive, f"aug_{i:04d}.tif")
        with rasterio.open(out_img_path, "w", **img_profile) as dst:
            for b in range(mixed_img.shape[2]):
                dst.write(mixed_img[:, :, b], b + 1)

        out_label_path = os.path.join(out_label_drive, f"aug_{i:04d}.tif")
        with rasterio.open(out_label_path, "w", **label_profile) as dst:
            dst.write(mask_binary, 1)

        print(f"[{i+1}/{n}] Saved -> {out_img_path}, {out_label_path}")


In [ ]:
target_paths = load_random_paths(Forest_Drive, n)
source_paths = load_random_paths(NonForest_Drive, n)

geom_transform = A.Compose(
    [
        A.HorizontalFlip(p=0.9),
        A.VerticalFlip(p=0.9),
        A.RandomRotate90(p=0.9),
    ],
    additional_targets={
        "raw": "image",
        "target": "image"
    }
)

for i, (target_path, source_path) in enumerate(zip(target_paths, source_paths)):
    # --- Load normalized (for CAM) ---
    s_img = CustomGeoDataset2([source_path], transform=None)[0]  # normalized (C,H,W)
    s_img = s_img.numpy().transpose(1, 2, 0)  # HWC in [0,1]

    # --- Load unnormalized (for blending) ---
    with rasterio.open(source_path) as src:
        s2_img = src.read().transpose(1, 2, 0).astype(np.float32)  # original values
        s2_img = cv2.resize(s2_img, (380, 380), interpolation=cv2.INTER_LINEAR)
        #src_profile = src.profile

    # --- Apply SAME augmentation to both normalized + unnormalized ---
    aug_out = geom_transform(image=s_img, raw=s2_img)
    s_aimg = aug_out["image"]   # normalized → for CAM
    s2_aimg = aug_out["raw"]    # unnormalized → for blending

    # --- Saliency mask from normalized ---
    s_aimg = np.transpose(s_aimg, (2, 0, 1))
    s_tensor = torch.tensor(s_aimg, dtype=torch.float32).to(DEVICE)
    mask = apply_cam_and_mask(model, s_tensor, device=DEVICE)  # (H,W)
    mask_binary = (mask > 0.25).astype(np.uint8)

    # --- Load and augment target ---
    with rasterio.open(target_path) as tgt:
        profile = tgt.profile
        t_img = tgt.read().transpose(1, 2, 0).astype(np.float32)

    t_aimg = geom_transform(image=t_img)["image"]

    # --- Blend in original value space ---
    mask_expanded = np.expand_dims(mask, axis=-1).astype(np.float32)
    s2_aimg = cv2.resize(s2_img, (256, 256), interpolation=cv2.INTER_LINEAR)
    t_aimg = cv2.resize(t_aimg, (256, 256), interpolation=cv2.INTER_LINEAR)
    mixed_img = mask_expanded * s2_aimg + (1 - mask_expanded) * t_aimg

    # --- Prepare profiles ---
    img_profile = profile.copy()
    img_profile.update({
        "count": mixed_img.shape[2],
        "dtype": "float32"
    })

    label_profile = profile.copy()
    label_profile.update({
        "count": 1,
        "dtype": "float32"
    })
    
    # --- Save results ---
    out_img_path = os.path.join(out_img_drive, f"aug_image_{i:04d}.tif")
    with rasterio.open(out_img_path, "w", **img_profile) as dst:
        for b in range(mixed_img.shape[2]):
            dst.write(mixed_img[:, :, b], b + 1)

    out_label_path = os.path.join(out_label_drive, f"aug_label_{i:04d}.tif")
    with rasterio.open(out_label_path, "w", **label_profile) as dst:
        dst.write(mask_binary, 1)

    print(f"[{i+1}/{n}] Saved -> {out_img_path}, {out_label_path}")
    